# Discrete sampling
In the last problem of the first session we sampled with given probabilities using prefix sums and binary search. The sampling code is the bottleneck of the whole process, its running times is 3 times higher than the code for uniform probabilities, and the difference would only increase for larger number of outcomes. In the next two problems we discuss two simple, but powerful techniques one can use to sample in time $O(1)$.

**Problem 2a.** Consider the problem of sampling with known probabilities $p_1,\ldots,p_d$. Suppose that you have a black-box that samples with probabilities $q_1,\ldots,q_d$ that are close to $p_1,\ldots,p_d$, say
$$ \forall_{i=1,\ldots,n} p_i \le (1+\varepsilon)q_i.$$

* How can you use this black-box to sample with probabilities $p_1,\ldots,p_d$? It is expected, that the running time of the algorithm would be non-deterministic.
* Prove that your algorithm is correct.
* Implement the algorithm and use it to give a faster implementation for **Problem 1c**.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('us_births_69_88.csv', usecols=["births"]).to_numpy().flatten()

S = sum(df)

d = len(df)

q = [ (1 / d) for k in range(d) ]

p = [ (df[k] / S) for k in range(d) ]

# this is the smallest number that meets the condition: p_i <= (1 + epsilon) * q_i for all i
epsilon = max([ p[k] / q[k] for k in range(d) ]) - 1

def blackBox():
    return np.random.randint(d)

def sample():
    i = blackBox()
    if np.random.rand() < p[i] / (q[i] * (1 + epsilon)):
        return i
    else:
        pass

def timesUntilRepetition():
    s = set()
    iterations = 0
    while True:
        iterations += 1
        x = sample()
        if x is None:
            continue
        if x not in s:
            s.add(x)
        else:
            return iterations

res = [ timesUntilRepetition() for k in range(100000)]
plt.hist(res)
plt.xlabel("number of times until first repetition")
plt.ylabel("number of repetitions")
plt.show()

Note that `sample()` consists of two steps: 
1. pick a number from blackBox (it returns $i$ with probability $q_i$)
2. decide if the result should be accepted (it accepts $i$ with probability $\frac{p_i}{q_i \cdot (1 + \varepsilon)}$)
   
Therefore, we can use Bayes theorem to calculate the probability:
$$ \mathbb{P}(\text{blackBox returns i | number got accepted}) \stackrel{\text{Bayes}}{=} \\ \frac{\mathbb{P}(\text{blackBox returns i})\cdot\mathbb{P}(\text{number got accepted | blackBox returns i})}{\sum_{j=1}^n \mathbb{P}(\text{blackBox returns j})\cdot\mathbb{P}(\text{number got accepted | blackBox returns j})} = \\ \frac{q_i \cdot \frac{p_i}{q_i(1+\varepsilon)}}{\sum_{j=1}^n q_j\cdot\frac{p_j}{q_j(1+\varepsilon)}} = \frac{p_i / (1 + \varepsilon)}{(\sum_{j=1}^n p_j) / (1 + \varepsilon)} = p_i$$
Thus, we are done.

**Problem 2b.** One of the reasons this implementation is not significantly faster than the one in **Problem 1c** , apart from $d$ being rather small, is that we are using Python's interpreter a bit too much, and Python is slow. One way around this is usually to use a library function - **searchsorted** is much faster than an equivalent code implemented in pure Python. But even if the functionality you need is not implemented in a lower level language as
a library function, there is still hope. You can try to implement it using optimized array algebra, for example using **numpy**. In this problem, your task is to rewrite the previous algorithm, so that the amount of *looping* is reduced to a minimum necessary. In particular, you should create a *vectorized* version of random dates generation (in bulk), while the actual search for duplicates remains a loop with a **set**. Here are some useful tips:
   * You can perform arithmetic, comparisons, etc. on **numpy** arrays.
   * You can generate whole **numpy** arrays of random numbers at once.
   * You can even perform parallel look-up like in the example below.

In [20]:
X = np.array([10,3,7])
I = np.array([1,1,2,2])
print(X[I])
X = np.array([[1,2],[3,4]])
I = np.array([0,0,1])
J = np.array([1,0,1])
print(X[I,J])

[3 3 7 7]
[2 1 4]


**Problem 2c (Squaring the histogram).** In this problem, we again want to sample with known probabilities $p_1,\ldots,p_n$, but this time we make no assumptions on $p_i$. Consider the following algorithm:
   * Let $V$ be the mean of $p_i$, i.e. $V=\frac{1}{n}$.
   * Create $n$ buckets, each with volume $V$, put each $p_i$ into a separate bucket.
   * Until there exists a bucket $A$ that is not full, find a bucket $B$ that overflows, and trasfer probability from $B$ to $A$ until $A$ is exactly full

Show that:
   * This algorithm always ends.
   * When it ends, each bucket contains pieces of exactly two $p_i$'s.

How to use the result of this algorithm to sample with probabilities $p_i$. Argue that your algorithm is correct and implement it. The sampling part should be *vectorized*. Use this algorithm to sample birthdates again, and test its efficiency.


**Problem 2d.** Show that the frequency histogram for empirical birthday frequencies can actually be computed exactly, and implement your idea. To this end, design a recurence relation using conditional probabilities and use dynamic programming.

**BONUS**. An alternative plotting library: plotly

In [ ]:
#install plotly with: pip install plotly
#you can run this shell command directly from Jupyter, by prefixing it with !
!pip install plotly

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np

init_notebook_mode(connected=True)

# example scatterplot

x = np.linspace(0, 1, 10)
y = x * x

iplot([go.Scatter(x=x, y=y, name="y=x^2"), go.Scatter(x=x, y=-y, name="y=-x^2")])

# example heatmap

iplot([go.Heatmap(z=[[10, 20, 30, 40],
                      [20, 30, 40, 50],
                      [30, 40, 50, 60]])])
